In [12]:
import pandas as pd
import re

In [13]:
#reads the data
with open('Raw_data_2023.txt', 'r', encoding='utf-8') as file:
    content = file.read()

In [14]:
df = pd.DataFrame([content],columns=['Content'])

df

,Content
0,"\nSoderblog\nSEEN, READ 2023\nMr/Dr Soderbergh..."


In [16]:
split_content= df.iloc[0,0].split('\n')
new_df = pd.DataFrame(split_content, columns=['Content'])


In [17]:
new_df = new_df.iloc[10:].reset_index(drop=True)

In [18]:
new_df["Dates"] = new_df["Content"].str.extract(r'(\d{2}/\d{2})')
new_df["Content"] = new_df["Content"].apply(lambda x: x[5:])
new_df["Content"] = new_df["Content"].str.replace(r' data-mce-fragment="1"', "")
new_df["Content"] = new_df["Content"].str.replace(r'(</em>,)[^,]*', r'\1', regex=True)

new_df

,Content,Dates
0,,NaN
1,"BELOW DECK, PRINCE ANDREW THE MUSICAL, THE BE...",01/01
2,"<em>Tomorrow, and Tomorrow, and Tomorrow</em>...",01/03
3,MADOFF: THE MONSTER OF WALL STREET (3),01/04
4,"<strong>ATHENA</strong>, MADOFF: THE MONSTER ...",01/05
...,...,...
263,"<em>Purple Hibiscus</em>,, <strong>NEITHER CO...",12/28
264,"<em>Candace Pert</em>,, HOUSE HUNTERS INTERNA...",12/29
265,"<em>Genealogy of a Murder</em>,",12/30
266,"<strong>ORIGIN</strong>, DAVE CHAPPELLE: THE ...",12/31


In [20]:
new_df = new_df.assign(Content = new_df.Content.str.split('</em>')).explode("Content")
new_df = new_df.assign(Content = new_df.Content.str.split('</strong>')).explode("Content")
new_df = new_df.assign(Content = new_df.Content.str.split(',')).explode("Content")

new_df

,Content,Dates
0,,NaN
1,BELOW DECK,01/01
1,PRINCE ANDREW THE MUSICAL,01/01
1,THE BEST OF COMEDY 2022,01/01
2,<em>Tomorrow,01/03
...,...,...
266,,12/31
266,DAVE CHAPPELLE: THE DREAMER,12/31
266,<strong>MICHAEL CLAYTON,12/31
266,,12/31


In [9]:
new_df = new_df[new_df["Content"] != ""]
new_df

,Content,Dates
1,BELOW DECK,01/01
1,PRINCE ANDREW THE MUSICAL,01/01
1,THE BEST OF COMEDY 2022,01/01
2,<em>Tomorrow,01/03
2,and Tomorrow,01/03
...,...,...
264,<strong>GRAN TURISMO,12/29
265,<em>Genealogy of a Murder,12/30
266,<strong>ORIGIN,12/31
266,DAVE CHAPPELLE: THE DREAMER,12/31


In [10]:
new_df["Bold"] = new_df["Content"].apply(lambda x: 1 if '<strong>' in x else 0)
new_df["Italic"] = new_df["Content"].apply(lambda x: 1 if '<em>' in x else 0)
new_df["Short"] = new_df["Content"].apply(lambda x: 1 if '*' in x else 0)
new_df

,Content,Dates,Bold,Italic,Short
1,BELOW DECK,01/01,0,0,0
1,PRINCE ANDREW THE MUSICAL,01/01,0,0,0
1,THE BEST OF COMEDY 2022,01/01,0,0,0
2,<em>Tomorrow,01/03,0,1,0
2,and Tomorrow,01/03,0,0,0
...,...,...,...,...,...
264,<strong>GRAN TURISMO,12/29,1,0,0
265,<em>Genealogy of a Murder,12/30,0,1,0
266,<strong>ORIGIN,12/31,1,0,0
266,DAVE CHAPPELLE: THE DREAMER,12/31,0,0,0


In [11]:
new_df["Content"] = new_df["Content"].str.replace("<em>", "")
new_df["Content"] = new_df["Content"].str.replace("<strong>", "")
new_df["AllCaps"] = new_df["Content"].apply(lambda x: 1 if x.isupper() else 0)
new_df

,Content,Dates,Bold,Italic,Short,AllCaps
1,BELOW DECK,01/01,0,0,1,1
1,PRINCE ANDREW THE MUSICAL,01/01,0,0,1,1
1,THE BEST OF COMEDY 2022,01/01,0,0,1,1
2,Tomorrow,01/03,0,1,1,0
2,and Tomorrow,01/03,0,0,1,0
...,...,...,...,...,...,...
264,GRAN TURISMO,12/29,1,0,1,1
265,Genealogy of a Murder,12/30,0,1,1,0
266,ORIGIN,12/31,1,0,1,1
266,DAVE CHAPPELLE: THE DREAMER,12/31,0,0,1,1


In [12]:
def has_quoted_string(text):
        # Check if there's any content between quotation marks
        pattern = r'\u201c([^\u201d]*)\u201d'
        return 1 if re.search(pattern, str(text)) else 0

new_df["Quoted"] = new_df["Content"].apply(lambda x: has_quoted_string(x))
new_df

,Content,Dates,Bold,Italic,Short,AllCaps,Quoted
1,BELOW DECK,01/01,0,0,1,1,0
1,PRINCE ANDREW THE MUSICAL,01/01,0,0,1,1,0
1,THE BEST OF COMEDY 2022,01/01,0,0,1,1,0
2,Tomorrow,01/03,0,1,1,0,0
2,and Tomorrow,01/03,0,0,1,0,0
...,...,...,...,...,...,...,...
264,GRAN TURISMO,12/29,1,0,1,1,0
265,Genealogy of a Murder,12/30,0,1,1,0,0
266,ORIGIN,12/31,1,0,1,1,0
266,DAVE CHAPPELLE: THE DREAMER,12/31,0,0,1,1,0


In [13]:
new_df.to_csv('Content2024.csv', index=False)